In [12]:
import urllib3
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import re
import datetime
import time
import os

import subprocess as cmd

#Projeto https://covidzero.com.br/


def LerArquivo(subdiretorio):
    x = datetime.datetime.now()
    data = x.strftime("%Y%m%d")
    arq = open(subdiretorio,'r', encoding='utf-8')
    #print(arq.read())
    lista = []
    for item in arq:
        #print(item)
        lista.append(str(item).strip())
    print(lista)
    arq.close()
    return lista
    
def gravarNoArquivoUrl(n1,n2,diretorio):
    x = datetime.datetime.now()
    data = x.strftime('%d-%m-%Y_%H')
    arquivo = open(diretorio+'\\covidZero_resultado_'+ data+'.csv', 'a',encoding='utf-8')
    arquivo.write('%s;%s\n' %(n1,n2))
    arquivo.flush()
    arquivo.close()

def crawl(paginas, profundidade,diretorio):
    x = datetime.datetime.now()
    data = x.strftime("%Y/%m/%d")
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    user_agent = {'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    for i in range(profundidade):
        novas_paginas = set()
        for pagina in paginas:
            http = urllib3.PoolManager(10,headers=user_agent)
            #http = urllib3.PoolManager()
            try:
                dados_pagina = http.request('GET',pagina)
            except:
                print('Erro ao abrir a página' + pagina)
                continue
            
            sopa = BeautifulSoup(dados_pagina.data,'lxml')
            
            bolsolinks = set()
            links = sopa.find_all('a')
            contador = 1
            for link in links:

                if ('title' in link.attrs):
                    title = str(link.get('title').lower())
                    url = urljoin(pagina, str(link.get('href')))

                    if title.__contains__('coronavirus') or title.__contains__('COVID-19') or title.__contains__('covid'):
                        if url[-1] == '/':
                            tamanho = len(url)
                            url = url[:tamanho-1]
                        bolsolinks.add(url)
                        print(url)
                        valida = -1
                        if valida == -1:
                            print('ok')

                if ('href' in link.attrs):
                    url = urljoin(pagina, str(link.get('href')))
                    
                    #if url != link.get('href'):
                        #print(url)
                        #print(link.get('href'))
                        
                    if url.find("'") != -1:
                        continue
                    
                    #print(url)
                    url = url.split('#')[0]
                    #print(url)
                    #print('\n')
                    
                    if url[0:4] == 'http' or url[0:5] == 'https':
                        novas_paginas.add(url)
                        if url.__contains__('coronavirus') or url.__contains__('COVID-19') or url.__contains__('covid'):
                            if url[-1] == '/':
                                tamanho = len(url)
                                url = url[:tamanho-1]
                            bolsolinks.add(url)
                            print(url)
                            valida = -1
                            if valida == -1:
                                print('ok')

            
                    contador = contador + 1
            for bolso in bolsolinks:
                gravarNoArquivoUrl(pagina,bolso,diretorio)
                    
            print(contador)
            
if __name__ == '__main__':
    ##Obs se for necessario podemos fazer pela rede tor, para nao entrarmos na black list dos sites
    path = '.\Estados'
    diretorio = os.listdir(path)
    #print(diretorio)
    for a in diretorio:
        #print(a)
        pathSub = path + '\\'+ a
        subdiretorio =  os.listdir(pathSub)
        for file in subdiretorio:
            if 'txt' in file:
                #print(file)
                fullpath = pathSub+'\\'+file
                #print(fullpath)
                #print(pathSub)
                sites = LerArquivo(fullpath)
                crawl(sites,1,pathSub)
    
    #Commit no Git
    cp = cmd.run("git add .", check=True, shell=True)
    cp = cmd.run(f'git commit -m "Atualizando"', check=True, shell=True)
    cp = cmd.run(f"git push origin crawler_sites -f", check=True, shell=True)

['https://g1.globo.com/al/alagoas/', 'https://www.alagoas24horas.com.br/', 'https://www.tnh1.com.br/', 'https://www.cadaminuto.com.br/', 'https://tribunahoje.com/', 'https://novoextra.com.br/', 'https://veja.abril.com.br/noticias-sobre/alagoas/', 'https://d.gazetadealagoas.com.br/']
https://g1.globo.com/al/alagoas/noticia/2020/03/24/sobe-para-10-o-numero-de-casos-confirmados-de-coronavirus-em-alagoas-diz-governador.ghtml
ok
https://g1.globo.com/al/alagoas/noticia/2020/03/24/morre-em-maceio-paciente-com-suspeita-de-coronavirus.ghtml
ok
https://g1.globo.com/al/alagoas/noticia/2020/03/24/morre-em-maceio-paciente-com-suspeita-de-coronavirus.ghtml
ok
https://g1.globo.com/al/alagoas/noticia/2020/03/24/medicos-e-enfermeiros-de-maceio-tiram-duvidas-sobre-coronavirus-em-call-center-da-saude.ghtml
ok
734
https://www.alagoas24horas.com.br/1278882/prefeitura-adota-medidas-tributarias-para-minimizar-impactos-da-covid-19
ok
https://www.alagoas24horas.com.br/1278882/prefeitura-adota-medidas-tributari

https://www.cadaminuto.com.br/noticia/356416/2020/03/24/coronavirus-alagoas-tem-10-casos-confirmados-e-numero-de-suspeitos-e-de-175
ok
https://www.cadaminuto.com.br/noticia/356410/2020/03/24/sesau-aguarda-testes-laboratoriais-para-confirmar-se-paciente-morreu-de-covid-19-em-al
ok
https://www.cadaminuto.com.br/noticia/356410/2020/03/24/sesau-aguarda-testes-laboratoriais-para-confirmar-se-paciente-morreu-de-covid-19-em-al
ok
https://minutosertao.cadaminuto.com.br/noticia/15245/2020/03/24/video-no-sertao-cidades-ficam-desertas-diante-da-pandemia-do-novo-coronavirus
ok
https://minutosertao.cadaminuto.com.br/noticia/15245/2020/03/24/video-no-sertao-cidades-ficam-desertas-diante-da-pandemia-do-novo-coronavirus
ok
https://www.cadaminuto.com.br/noticia/356419/2020/03/24/devido-a-pandemia-de-coronavirus-proximas-etapas-do-concurso-da-sefaz-sao-adiadas
ok
https://www.cadaminuto.com.br/noticia/356415/2020/03/24/covid-19-resistencia-leva-mpe-al-a-orientar-policia-e-sec-de-saude-a-garantir-isolamen

https://novoextra.com.br/noticias/entretenimento/2020/03/54627-coronavirus-aumenta-uso-do-whatsapp-e-facebook-messenger
ok
https://novoextra.com.br/opiniao/artigos-de-elias-fragoso/2020/03/130-o-coronavirus-e-ameaca-a-economia-global
ok
https://novoextra.com.br/noticias/coronavirus
ok
https://novoextra.com.br/noticias/coronavirus/2020/03/54693-alagoas-registra-dez-casos-confirmados-de-coronavirus
ok
https://novoextra.com.br/noticias/coronavirus/2020/03/54692-governadores-e-prefeitos-tem-poderes-para-fechar-divisas-decide-stf
ok
https://novoextra.com.br/noticias/coronavirus/2020/03/54687-saiba-onde-tem-e-o-preco-do-alcool-gel-em-maceio
ok
https://novoextra.com.br/noticias/coronavirus/2020/03/54693-alagoas-registra-dez-casos-confirmados-de-coronavirus
ok
https://novoextra.com.br/noticias/coronavirus/2020/03/54681-velorios-e-enterros-terao-duracao-maxima-de-tres-horas-em-maceio
ok
https://novoextra.com.br/noticias/coronavirus/2020/03/54685-covid-19-quarentena-e-escolas-fechadas-sao-combin

https://g1.globo.com/pe/pernambuco/noticia/2020/03/24/secretarios-de-saude-do-nordeste-dizem-que-boslonaro-desfaz-todo-o-esforco-para-combater-coronavirus.ghtml
ok
https://g1.globo.com/pe/pernambuco/noticia/2020/03/24/pernambuco-tem-mais-dois-pacientes-curados-e-nao-confirma-novos-casos-do-novo-coronavirus.ghtml
ok
https://g1.globo.com/pe/pernambuco/noticia/2020/03/24/pernambuco-tem-mais-dois-pacientes-curados-e-nao-confirma-novos-casos-do-novo-coronavirus.ghtml
ok
https://g1.globo.com/pe/pernambuco/noticia/2020/03/24/recife-rastreia-700-mil-celulares-para-monitorar-isolamento-social-e-direcionar-acoes-contra-coronavirus.ghtml
ok
https://g1.globo.com/pe/pernambuco/noticia/2020/03/18/coronavirus-em-pernambuco-aulas-suspensas-restricao-a-servicos-veja-situacao-apos-medidas-contra-a-pandemia.ghtml
ok
https://g1.globo.com/pe/pernambuco/noticia/2020/03/24/deputados-destinam-verbas-de-emendas-ao-enfrentamento-do-coronavirus-e-aprovam-calamidade.ghtml
ok
https://g1.globo.com/pe/pernambuco/not

https://noticias.r7.com/saude/coronavirus
ok
233
https://tvjornal.ne10.uol.com.br/noticias/2020/03/24/pernambuco-registra-cinco-pacientes-recuperados-de-coronavirus-185883
ok
https://tvjornal.ne10.uol.com.br/noticias/2020/03/24/ministerio-da-saude-registra-mais-de-dois-mil-casos-de-coronavirus-e-46-mortes-no-brasil-185882
ok
https://tvjornal.ne10.uol.com.br/noticias/2020/03/24/por-conta-do-coronavirus-saiba-os-novos-horarios-da-caixa-economica-e-do-banco-do-brasil-185870
ok
https://tvjornal.ne10.uol.com.br/noticias/2020/03/24/celpe-e-proibida-de-cortar-energia-durante-pandemia-do-coronavirus-sob-pena-de-multa-185865
ok
https://tvjornal.ne10.uol.com.br/noticias/2020/03/24/coronavirus-governo-de-pernambuco-proibe-aglomeracao-em-onibus-e-terminais-integrados-185860
ok
https://tvjornal.ne10.uol.com.br/o-povo-natv/2020/03/23/pescador-aproveita-boa-pescaria-e-faz-doacao-para-ajudar-os-moradores-de-ipojuca-durante-a-quarentena-do-coronavirus-185815
ok
https://tvjornal.ne10.uol.com.br/noticias

https://oglobo.globo.com/sociedade/coronavirus
ok
https://oglobo.globo.com/sociedade/coronavirus-servico
ok
https://oglobo.globo.com/brasil/governadores-preparam-reacao-conjunta-bolsonaro-mantem-restricoes-para-combater-coronavirus-24326651
ok
https://infograficos.oglobo.globo.com/sociedade/numeros-do-coronavirus-no-brasil-e-no-mundo.html
ok
https://infograficos.oglobo.globo.com/sociedade/numeros-do-coronavirus-no-brasil-e-no-mundo.html
ok
https://blogs.oglobo.globo.com/guga-chacra/post/bolsonaro-adota-postura-oposta-de-israel-no-combate-ao-coronavirus.html
ok
https://blogs.oglobo.globo.com/guga-chacra/post/bolsonaro-adota-postura-oposta-de-israel-no-combate-ao-coronavirus.html
ok
https://oglobo.globo.com/sociedade/coronavirus/para-especialistas-bolsonaro-foi-irresponsavel-ao-pedir-fim-da-quarentena-contra-coronavirus-24326709
ok
https://oglobo.globo.com/esportes/historico-de-atleta-veja-lista-de-profissionais-do-esporte-afetados-pela-covid-19-24326447
ok
https://oglobo.globo.com/espor

https://meiahora.ig.com.br/geral/2020/03/5888458-bolsonaro-ataca--midia---apoia-o-fim-de-isolamento-e-diz-que--90--da-populacao--nao-tera-coronavirus.html
ok
https://meiahora.ig.com.br/geral/2020/03/5888245-numero-de-mortes-por-coronavirus-sobe-para-seis-no-rio.html
ok
https://meiahora.ig.com.br/geral/2020/03/5888274-mortes-pelo-coronavirus-chegam-a-46-no-brasil--pais-ultrapassa-2-mil-casos.html
ok
https://meiahora.ig.com.br/geral/mundo-e-tecnologia/2020/03/5888408-situacao-critica--franca-supera-os-mil-mortos-por-coronavirus.html
ok
https://meiahora.ig.com.br/geral/2020/03/5888167-sem-turismo-por-conta-do-coronavirus--bondinho-pao-de-acucar-recebe-visitantes-ilustres.html
ok
https://meiahora.ig.com.br/geral/2020/03/5888406-nao-e-so-o-coronavirus---brasil-registra-200-casos-de-tuberculose-por-dia.html
ok
https://meiahora.ig.com.br/esportes/2020/03/5887886-primeiro-ministro-japones-anuncia-adiamento-da-olimpiada-para-2021-por-conta-do-coronavirus.html
ok
https://meiahora.ig.com.br/espor

http://www.tribunadonorte.com.br/noticia/covid-19-governo-do-estado-proa-be-que-rede-privada-negue-atendimento/475672
ok
236
http://www.tribunadenoticias.com.br/2020/03/por-coronavirus-aneel-suspende-cortes.html
ok
http://www.tribunadenoticias.com.br/2020/03/por-coronavirus-aneel-suspende-cortes.html
ok
http://www.tribunadenoticias.com.br/2020/03/por-coronavirus-aneel-suspende-cortes.html
ok
http://www.tribunadenoticias.com.br/2020/03/por-coronavirus-aneel-suspende-cortes.html
ok
http://www.tribunadenoticias.com.br/2020/03/deputado-do-rn-diagnosticado-com-covid.html
ok
http://www.tribunadenoticias.com.br/2020/03/deputado-do-rn-diagnosticado-com-covid.html
ok
http://www.tribunadenoticias.com.br/2020/03/deputado-do-rn-diagnosticado-com-covid.html
ok
http://www.tribunadenoticias.com.br/2020/03/deputado-do-rn-diagnosticado-com-covid.html
ok
http://www.tribunadenoticias.com.br/2020/03/rn-tem-mais-um-caso-confirmado-de-covid.html
ok
http://www.tribunadenoticias.com.br/2020/03/rn-tem-mais-um-

CompletedProcess(args="echo 'I will automate everythig with Python.'", returncode=0)